#**IMPLEMENTATION OF WLASL PAPER**

**Some Imports**

In [ ]:
import os 
import numpy as np
import cv2
import math
import shutil
import random
import json
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Lambda
# from tensorflow.keras.utils.data_utils import get_file
from tensorflow.keras import backend as K
from tensorflow.python.ops.distributions.uniform import Uniform
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt


In [ ]:
%cd /content/drive/MyDrive/WLASL/code/I3D/

/content/drive/MyDrive/WLASL/code/I3D


**Downloading and Preprocessing Of Dataset**

In [ ]:
!git clone https://github.com/dxli94/WLASL
!pip install youtube-dl 

In [ ]:
!python video_downloader.py
!python preprocess.py

/content/drive/My Drive/WLASL/start_kit


Check The Number Of Videos

In [ ]:
print(len(os.listdir("raw_videos/")))
print(len(os.listdir("raw_videos_mp4/")))
print(len(os.listdir("videos/")))

**Split WLASL 100 words into Train ,Valid ,Test**






In [ ]:
%cd /content/drive/MyDrive/WLASL/data100/
!pwd

/content/drive/MyDrive/WLASL/data100
/content/drive/MyDrive/WLASL/data100


In [ ]:
content = json.load(open('/content/drive/MyDrive/WLASL/code/I3D/preprocess/nslt_100.json'))


if not os.path.exists('train'):
    os.mkdir('train')
if not os.path.exists('val'):
    os.mkdir('val')
if not os.path.exists('test'):
    os.mkdir('test')

for key,value in content.items():
    mode = value['subset']
    video_id=key
    label = str(value['action'][0])
    # print(mode , video_id , label)
    if not os.path.exists(os.path.join(mode,label)):
        os.mkdir(os.path.join(mode,label))
    src = os.path.join('/content/drive/MyDrive/WLASL/start_kit/videos',video_id +'.mp4')
    dest = os.path.join(mode,label,video_id +'.mp4')
    if os.path.exists(dest):
        continue
    if not os.path.exists(src):
        continue
    # crop_video_file(instance['bbox'],os.path.join(root,src),os.path.join(root,dest),video_id +'.mp4')
    shutil.copy(src,dest)

Check if data100 is complete

In [ ]:
count=0
data_path='/content/drive/MyDrive/WLASL/data100'
folders = os.listdir(data_path)
for folder in folders:
  folder_mode = os.listdir(os.path.join(data_path,folder))
  for folder_class in folder_mode:
    num_of_videos = len(os.listdir(os.path.join(data_path,folder,folder_class)))
    count = count + num_of_videos
print(count)

1797


In [ ]:
content = json.load(open('/content/drive/MyDrive/WLASL/code/I3D/preprocess/nslt_100.json'))
len(content)

2038

**Split WLASL 2000 words into Train ,Valid ,Test**


In [ ]:
%cd /content/drive/MyDrive/WLASL/data2000/
!pwd

/content/drive/MyDrive/WLASL/data2000
/content/drive/MyDrive/WLASL/data2000


In [ ]:
content = json.load(open('/content/drive/MyDrive/WLASL/code/I3D/preprocess/nslt_2000.json'))


if not os.path.exists('train'):
    os.mkdir('train')
if not os.path.exists('val'):
    os.mkdir('val')
if not os.path.exists('test'):
    os.mkdir('test')

for key,value in content.items():
    mode = value['subset']
    video_id=key
    label = str(value['action'][0])
    # print(mode , video_id , label)
    if not os.path.exists(os.path.join(mode,label)):
        os.mkdir(os.path.join(mode,label))
    src = os.path.join('/content/drive/MyDrive/WLASL/start_kit/videos',video_id +'.mp4')
    dest = os.path.join(mode,label,video_id +'.mp4')
    if os.path.exists(dest):
        continue
    if not os.path.exists(src):
        continue
    # crop_video_file(instance['bbox'],os.path.join(root,src),os.path.join(root,dest),video_id +'.mp4')
    shutil.copy(src,dest)



Check if data2000 is complete

In [ ]:
count=0
data_path='/content/drive/MyDrive/WLASL/data2000'
folders = os.listdir(data_path)
for folder in folders:
  folder_mode = os.listdir(os.path.join(data_path,folder))
  for folder_class in folder_mode:
    num_of_videos = len(os.listdir(os.path.join(data_path,folder,folder_class)))
    count = count + num_of_videos
print(count)

19049


In [ ]:
content = json.load(open('/content/drive/MyDrive/WLASL/code/I3D/preprocess/nslt_2000.json'))
len(content)

21095

##**Convert WLASL 100 Words To Frames data100_frames**

if you will load data as videos using generator , skip this part!

In [ ]:
# path = 'data100_frames/train/'
# for i in range(100):
#   os.mkdir(path+str(i))

In [ ]:
# path = 'data100_frames/val/'
# for i in range(100):
#   os.mkdir(path+str(i))

In [ ]:
# path = 'data100_frames/test/'
# for i in range(100):
#   os.mkdir(path+str(i))

In [ ]:
def video_to_frames(video_path, size=(256,256)):
  """
  video_path -> str, path to video.
  size -> (int, int), width, height.
  """

  cap = cv2.VideoCapture(video_path)
  frames = []
  
  while True:
      ret, frame = cap.read()
  
      if ret:
          if size:
              frame = cv2.resize(frame, size)
          frames.append(frame)
      else:
          break

  cap.release()
  return np.array(frames)

def random_crop(frames,out_size=(224,224)):
  t,h,w,c = frames.shape
  th,tw = out_size
  if w == tw and h == th :
      i , j = 0 , 0
  else:
      i = random.randint(0,(h-th)) if h != th else 0
      j = random.randint(0,(w-tw)) if w != tw else 0
  
  return frames[:, i:i+th, j:j+tw, :] 

def center_crop(frames,out_size=(224,224)):
  t,h,w,c = frames.shape
  th,tw = out_size
  i = int(np.round((h - th) / 2.))
  j = int(np.round((w - tw) / 2.))
  return imgs[:, i:i+th, j:j+tw, :]

def load_rgb_frames_from_video(video_path, start, num, resize=(256, 256)):

  
  vidcap = cv2.VideoCapture(video_path)

  frames = []

  total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

  vidcap.set(cv2.CAP_PROP_POS_FRAMES, start)
  for offset in range(min(num, int(total_frames - start))):
      success, img = vidcap.read()
      if success:
        w, h, c = img.shape
        if w < 226 or h < 226:
            d = 226. - min(w, h)
            sc = 1 + d / min(w, h)
            img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)

        if w > 256 or h > 256:
            img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))

        # img = (img / 255.) * 2 - 1

        frames.append(img)
  return np.array(frames)
def make_dataset(split_file, video_path, mode,vid):
  '''
  split file : path of nslt_100.json
  mode : flow or rgb
  num_classes : should be for now 100
  '''
  
  with open(split_file, 'r') as f:
      data = json.load(f)

  count_skipping = 0
  

  src = 0

  if not os.path.exists(video_path):
      return ()

  num_frames = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT))

  if mode == 'flow':
      num_frames = num_frames // 2

  if num_frames - 0 < 9:
      print("Skip video ", vid)
      count_skipping += 1
      return ()

  # label = np.zeros((num_classes, num_frames), np.float32)

  # for l in range(num_frames):
  #     c_ = data[vid]['action'][0]
  #     label[c_][l] = 1
  if count_skipping :
      print("skipped")

  if len(vid) == 5:
      return (vid, src, 0, data[vid]['action'][2] - data[vid]['action'][1])
  elif len(vid) == 6:  ## sign kws instances
      return (vid, src, data[vid]['action'][1], data[vid]['action'][2] - data[vid]['action'][1])

**(Train)**

In [ ]:
split_file = "code/I3D/preprocess/nslt_100.json"
mode = "rgb"
paths = "data100/train/"
dest = "data100_frames/train/"
folders = os.listdir(paths)   
for folder in folders:
    videos = os.listdir(os.path.join(paths,folder))
    for video in videos:
        video_path = os.path.join(paths,folder,video)
        vid = video[:-4]
        info  = make_dataset(split_file , video_path, mode , vid)
        if info:
            vid, src, start_frame , nf = info
            total_frames = 64

            try:
                start_f = random.randint(0, nf - total_frames - 1) + start_frame
            except ValueError:
                start_f = start_frame

            imgs = load_rgb_frames_from_video(video_path, start_f, total_frames)
            frames = random_crop(imgs)
            count = 0
            if not os.path.exists(os.path.join(dest,folder,vid)):
              os.mkdir(os.path.join(dest,folder,vid))
            for frame in frames:
                count +=1
                if os.path.exists(os.path.join(dest,folder,vid , '{}.jpg').format(count)):
                  # print(os.path.join(dest,folder,vid , '{}.jpg').format(count) + " already exists")
                  continue
                else :
                  print("Saving" + os.path.join(dest,folder,vid , '{}.jpg').format(count))
                  cv2.imwrite(os.path.join(dest,folder,vid , '{}.jpg').format(count),frame)

Savingdata100_frames/train/55/11775/60.jpg


**(Val)**

In [ ]:
split_file = "code/I3D/preprocess/nslt_100.json"
mode = "rgb"
paths = "data100/val/"
dest = "data100_frames/val/"
folders = os.listdir(paths)   
for folder in folders:
    videos = os.listdir(os.path.join(paths,folder))
    for video in videos:
        video_path = os.path.join(paths,folder,video)
        vid = video[:-4]
        info  = make_dataset(split_file , video_path, mode , vid)
        if info:
            vid, src, start_frame , nf = info
            total_frames = 64

            try:
                start_f = random.randint(0, nf - total_frames - 1) + start_frame
            except ValueError:
                start_f = start_frame

            imgs = load_rgb_frames_from_video(video_path, start_f, total_frames)
            frames = center_crop(imgs)
            count = 0
            if not os.path.exists(os.path.join(dest,folder,vid)):
              os.mkdir(os.path.join(dest,folder,vid))
            for frame in frames:
                count +=1
                if os.path.exists(os.path.join(dest,folder,vid , '{}.jpg').format(count)):
                  #  print(os.path.join(dest,folder,vid , '{}.jpg').format(count) + " already exists")
                  continue
                else :
                  print("Saving" + os.path.join(dest,folder,vid , '{}.jpg').format(count))
                  cv2.imwrite(os.path.join(dest,folder,vid , '{}.jpg').format(count),frame)

**(Test)**

In [ ]:
split_file = "code/I3D/preprocess/nslt_100.json"
mode = "rgb"
paths = "data100/test/"
dest = "data100_frames/test/"
folders = os.listdir(paths)   
for folder in folders:
    videos = os.listdir(os.path.join(paths,folder))
    for video in videos:
        video_path = os.path.join(paths,folder,video)
        vid = video[:-4]
        info  = make_dataset(split_file , video_path, mode , vid)
        if info:
            vid, src, start_frame , nf = info
            total_frames = 64

            try:
                start_f = random.randint(0, nf - total_frames - 1) + start_frame
            except ValueError:
                start_f = start_frame

            imgs = load_rgb_frames_from_video(video_path, start_f, total_frames)
            frames = center_crop(imgs)
            count = 0
            if not os.path.exists(os.path.join(dest,folder,vid)):
              os.mkdir(os.path.join(dest,folder,vid))
            for frame in frames:
                count +=1
                if os.path.exists(os.path.join(dest,folder,vid , '{}.jpg').format(count)):
                  # print(os.path.join(dest,folder,vid , '{}.jpg').format(count) + " already exists")
                  continue
                else :
                  print("Saving" + os.path.join(dest,folder,vid , '{}.jpg').format(count))
                  cv2.imwrite(os.path.join(dest,folder,vid , '{}.jpg').format(count),frame)

**Check if data100_frames is completed**

In [ ]:
count=0
data_path='/content/drive/MyDrive/WLASL/data100_frames'
folders = os.listdir(data_path)
for folder in folders:
  folder_mode = os.listdir(os.path.join(data_path,folder))
  for folder_class in folder_mode:
    num_of_videos = len(os.listdir(os.path.join(data_path,folder,folder_class)))
    count = count + num_of_videos
print(count)

1797


**Apply Padding**

In [ ]:
def padding(folder_path):
    ref_frames = ['1.jpg','2.jpg', '3.jpg' ,'4.jpg','5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg','10.jpg',
     '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg','20.jpg',
     '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', 
     '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg','40.jpg', 
     '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg', 
     '51.jpg', '52.jpg', '53.jpg', '54.jpg', '55.jpg', '56.jpg', '57.jpg', '58.jpg', '59.jpg', '60.jpg'
     '61.jpg' ,'62.jpg' ,'63.jpg' ,'64.jpg']
    sorted_frames = []     
    frames = os.listdir(folder_path)
    for x in ref_frames:
        if x in frames:
            sorted_frames.append(x)
        else:
            break
    nf = len(sorted_frames)
    if nf != 64 :
        print("Saving at ...."+folder_path)
        num_padding = 64 - nf
        prob = np.random.random_sample()
        if prob > 0.5:
            pad_img = cv2.imread(os.path.join(folder_path,sorted_frames[0]))
            pad = np.tile(np.expand_dims(pad_img, axis=0), (num_padding, 1, 1, 1))
            
        else:
            pad_img = cv2.imread(os.path.join(folder_path,sorted_frames[-1]),1)
            pad = np.tile(np.expand_dims(pad_img, axis=0), (num_padding, 1, 1, 1))
        count = nf + 1
        for frame in pad:
            # print("Saving...."+os.path.join(folder_path,str(count))+'.jpg')
            cv2.imwrite(os.path.join(folder_path,str(count))+'.jpg',frame )
            count += 1

**(Train)**

In [ ]:
paths = "data100_frames/train/"
folders = os.listdir(paths)   
for folder in folders:
  print(folder)
  videos = os.listdir(os.path.join(paths,folder))
  for video in videos:
    video_path = os.path.join(paths,folder,video)
    padding(video_path)

**(Val)**

In [ ]:
paths = "data100_frames/val/"
folders = os.listdir(paths)   
for folder in folders:
  print(folder)
  videos = os.listdir(os.path.join(paths,folder))
  for video in videos:
    video_path = os.path.join(paths,folder,video)
    padding(video_path)

**(Test)**

In [ ]:
paths = "data100_frames/test/"
folders = os.listdir(paths)   
for folder in folders:
  print(folder)
  videos = os.listdir(os.path.join(paths,folder))
  for video in videos:
    video_path = os.path.join(paths,folder,video)
    padding(video_path)

**Check all videos are 64 frames**

In [ ]:
paths = "data100_frames/"
modes = os.listdir(paths)
for mode in modes:
  folders = os.listdir(os.path.join(paths,mode))
  for folder in folders:
    videos = os.listdir(os.path.join(paths,mode,folder))
    for video in videos:
      video_path = os.path.join(paths,mode,folder,video)
      nf = len(os.listdir(video_path))
      if nf != 64:
        print('Mode: ' + mode + '......' + 'Class: ' + folder + '......' + 'Video_id: ' + video + '......' 'No. Frames: ' + nf)

##**Modeling I3D**

In [ ]:
WEIGHTS = {
    'flow_imagenet_and_kinetics':
        'flow_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels.h5',
    'flow_imagenet_and_kinetics_no_top':
        'flow_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels_no_top.h5',
    'rgb_imagenet_and_kinetics':
        'rgb_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels.h5',
    'rgb_imagenet_and_kinetics_no_top':
        'rgb_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels_no_top.h5',
}


def Unit_3d(x,
            filters,
            num_frames,
            num_row,
            num_col,
            padding='same',
            strides=(1, 1, 1),
            use_bias=False,
            use_activation_fn=True,
            use_bn=True,
            name=None):
    """
    Utility function to apply conv3d + BN.
    :return: Output tensor after applying `Conv3D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None

    layer = Conv3D(
        filters, (num_frames, num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=use_bias,
        name=conv_name)(x)

    if use_bn:
        layer = BatchNormalization(axis=4, scale=False, name=bn_name)(layer)

    if use_activation_fn:
        layer = Activation('relu', name=name)(layer)

    return layer


def PreTrainedInception3d(include_top=True,
                          pretrained_weights='rgb_imagenet_and_kinetics',
                          input_shape=None,
                          dropout_prob=0.0,
                          endpoint_logit=True,
                          classes=400):
    """
    Instantiates the Inception i3D Inception v1 architecture.
    :return: Inception i3D model model.
    """
    if not include_top:
        pretrained_weights = pretrained_weights + '_no_top'

    if pretrained_weights not in WEIGHTS:
        raise ValueError('in valid weight name')

    input_shape = input_shape
    img_input = Input(shape=input_shape)
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 4

    # Downsampling via convolution (spatial and temporal)
    x = Unit_3d(img_input, 64, 7, 7, 7, strides=(2, 2, 2), padding='same', name='Conv3d_1a_7x7')

    # Downsampling (spatial only)
    x = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same', name='MaxPool2d_2a_3x3')(x)
    x = Unit_3d(x, 64, 1, 1, 1, strides=(1, 1, 1), padding='same', name='Conv3d_2b_1x1')
    x = Unit_3d(x, 192, 3, 3, 3, strides=(1, 1, 1), padding='same', name='Conv3d_2c_3x3')

    # Downsampling (spatial only)
    x = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same', name='MaxPool2d_3a_3x3')(x)

    # Mixed 3b
    branch_0 = Unit_3d(x, 64, 1, 1, 1, padding='same', name='Conv3d_3b_0a_1x1')

    branch_1 = Unit_3d(x, 96, 1, 1, 1, padding='same', name='Conv3d_3b_1a_1x1')
    branch_1 = Unit_3d(branch_1, 128, 3, 3, 3, padding='same', name='Conv3d_3b_1b_3x3')

    branch_2 = Unit_3d(x, 16, 1, 1, 1, padding='same', name='Conv3d_3b_2a_1x1')
    branch_2 = Unit_3d(branch_2, 32, 3, 3, 3, padding='same', name='Conv3d_3b_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_3b_3a_3x3')(x)
    branch_3 = Unit_3d(branch_3, 32, 1, 1, 1, padding='same', name='Conv3d_3b_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_3b')

    # Mixed 3c
    branch_0 = Unit_3d(x, 128, 1, 1, 1, padding='same', name='Conv3d_3c_0a_1x1')

    branch_1 = Unit_3d(x, 128, 1, 1, 1, padding='same', name='Conv3d_3c_1a_1x1')
    branch_1 = Unit_3d(branch_1, 192, 3, 3, 3, padding='same', name='Conv3d_3c_1b_3x3')

    branch_2 = Unit_3d(x, 32, 1, 1, 1, padding='same', name='Conv3d_3c_2a_1x1')
    branch_2 = Unit_3d(branch_2, 96, 3, 3, 3, padding='same', name='Conv3d_3c_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_3c_3a_3x3')(x)
    branch_3 = Unit_3d(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_3c_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_3c')

    # Downsampling (spatial and temporal)
    x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2), padding='same', name='MaxPool2d_4a_3x3')(x)

    # Mixed 4b
    branch_0 = Unit_3d(x, 192, 1, 1, 1, padding='same', name='Conv3d_4b_0a_1x1')

    branch_1 = Unit_3d(x, 96, 1, 1, 1, padding='same', name='Conv3d_4b_1a_1x1')
    branch_1 = Unit_3d(branch_1, 208, 3, 3, 3, padding='same', name='Conv3d_4b_1b_3x3')

    branch_2 = Unit_3d(x, 16, 1, 1, 1, padding='same', name='Conv3d_4b_2a_1x1')
    branch_2 = Unit_3d(branch_2, 48, 3, 3, 3, padding='same', name='Conv3d_4b_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4b_3a_3x3')(x)
    branch_3 = Unit_3d(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4b_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4b')

    # Mixed 4c
    branch_0 = Unit_3d(x, 160, 1, 1, 1, padding='same', name='Conv3d_4c_0a_1x1')

    branch_1 = Unit_3d(x, 112, 1, 1, 1, padding='same', name='Conv3d_4c_1a_1x1')
    branch_1 = Unit_3d(branch_1, 224, 3, 3, 3, padding='same', name='Conv3d_4c_1b_3x3')

    branch_2 = Unit_3d(x, 24, 1, 1, 1, padding='same', name='Conv3d_4c_2a_1x1')
    branch_2 = Unit_3d(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4c_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4c_3a_3x3')(x)
    branch_3 = Unit_3d(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4c_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4c')

    # Mixed 4d
    branch_0 = Unit_3d(x, 128, 1, 1, 1, padding='same', name='Conv3d_4d_0a_1x1')

    branch_1 = Unit_3d(x, 128, 1, 1, 1, padding='same', name='Conv3d_4d_1a_1x1')
    branch_1 = Unit_3d(branch_1, 256, 3, 3, 3, padding='same', name='Conv3d_4d_1b_3x3')

    branch_2 = Unit_3d(x, 24, 1, 1, 1, padding='same', name='Conv3d_4d_2a_1x1')
    branch_2 = Unit_3d(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4d_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4d_3a_3x3')(x)
    branch_3 = Unit_3d(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4d_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4d')

    # Mixed 4e
    branch_0 = Unit_3d(x, 112, 1, 1, 1, padding='same', name='Conv3d_4e_0a_1x1')

    branch_1 = Unit_3d(x, 144, 1, 1, 1, padding='same', name='Conv3d_4e_1a_1x1')
    branch_1 = Unit_3d(branch_1, 288, 3, 3, 3, padding='same', name='Conv3d_4e_1b_3x3')

    branch_2 = Unit_3d(x, 32, 1, 1, 1, padding='same', name='Conv3d_4e_2a_1x1')
    branch_2 = Unit_3d(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4e_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4e_3a_3x3')(x)
    branch_3 = Unit_3d(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4e_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4e')

    # Mixed 4f
    branch_0 = Unit_3d(x, 256, 1, 1, 1, padding='same', name='Conv3d_4f_0a_1x1')

    branch_1 = Unit_3d(x, 160, 1, 1, 1, padding='same', name='Conv3d_4f_1a_1x1')
    branch_1 = Unit_3d(branch_1, 320, 3, 3, 3, padding='same', name='Conv3d_4f_1b_3x3')

    branch_2 = Unit_3d(x, 32, 1, 1, 1, padding='same', name='Conv3d_4f_2a_1x1')
    branch_2 = Unit_3d(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_4f_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4f_3a_3x3')(x)
    branch_3 = Unit_3d(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_4f_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4f')

    # Downsampling (spatial and temporal)
    x = MaxPooling3D((2, 2, 2), strides=(2, 2, 2), padding='same', name='MaxPool2d_5a_2x2')(x)

    # Mixed 5b
    branch_0 = Unit_3d(x, 256, 1, 1, 1, padding='same', name='Conv3d_5b_0a_1x1')

    branch_1 = Unit_3d(x, 160, 1, 1, 1, padding='same', name='Conv3d_5b_1a_1x1')
    branch_1 = Unit_3d(branch_1, 320, 3, 3, 3, padding='same', name='Conv3d_5b_1b_3x3')

    branch_2 = Unit_3d(x, 32, 1, 1, 1, padding='same', name='Conv3d_5b_2a_1x1')
    branch_2 = Unit_3d(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_5b_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_5b_3a_3x3')(x)
    branch_3 = Unit_3d(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_5b_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_5b')

    # Mixed 5c
    branch_0 = Unit_3d(x, 384, 1, 1, 1, padding='same', name='Conv3d_5c_0a_1x1')

    branch_1 = Unit_3d(x, 192, 1, 1, 1, padding='same', name='Conv3d_5c_1a_1x1')
    branch_1 = Unit_3d(branch_1, 384, 3, 3, 3, padding='same', name='Conv3d_5c_1b_3x3')

    branch_2 = Unit_3d(x, 48, 1, 1, 1, padding='same', name='Conv3d_5c_2a_1x1')
    branch_2 = Unit_3d(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_5c_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_5c_3a_3x3')(x)
    branch_3 = Unit_3d(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_5c_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_5c')

    if include_top:
        # Classification block
        x = AveragePooling3D((2, 7, 7), strides=(1, 1, 1), padding='valid', name='global_avg_pool')(x)
        x = Dropout(dropout_prob)(x)

        x = Unit_3d(x, classes, 1, 1, 1, padding='same',
                    use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')

        num_frames_remaining = int(x.shape[1])
        x = Reshape((num_frames_remaining, classes))(x)

        # logits (raw scores for each class)
        x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
                   output_shape=lambda s: (s[0], s[2]))(x)

        if not endpoint_logit:
            x = Activation('softmax', name='prediction')(x)
    else:
        h = int(x.shape[2])
        w = int(x.shape[3])
        x = AveragePooling3D((2, h, w), strides=(1, 1, 1), padding='valid', name='global_avg_pool')(x)

    inputs = img_input
    # create model
    model = Model(inputs, x, name='i3d_inception')

    # load weights
    model.load_weights(WEIGHTS[pretrained_weights])
    model.trainable = False

    return model


def TopLayer(input_shape, classes, dropout_prob):
    inputs = Input(shape=input_shape, name="input")
    x = Dropout(dropout_prob)(inputs)
    x = Unit_3d(x, classes, 1, 1, 1, padding='same',
                  use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')
    num_frames_remaining = int(x.shape[1])
    x = Reshape((num_frames_remaining, classes))(x)
    x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
               output_shape=lambda s: (s[0], s[2]))(x)
    x = Activation('softmax', name='prediction')(x)
    final_model = Model(inputs=inputs, outputs=x, name="i3d_top")
    return final_model


def add_top_layer(base_model: Model, classes: int, dropout_prob: float):
    top_layer = TopLayer(base_model.output_shape[1:], classes, dropout_prob)
    x = base_model.output
    predictions = top_layer(x)
    new_model = Model(inputs=base_model.input, outputs=predictions, name="i3d_with_top")
    return new_model

def layers_freeze(model, leave_last=50):
    print("Freezing %d layers of %d in Model %s" % (len(model.layers)-leave_last, len(model.layers), model.name))
    for layer in model.layers[:-leave_last]:
        layer.trainable = False
    for layer in model.layers[-leave_last:]:
        layer.trainable = True
    return model

In [ ]:
def random_crop(frames,out_size=(224,224)):
    t,h,w,c = frames.shape
    th,tw = out_size
    if w == tw and h == th :
        i , j = 0 , 0
    else:
        i = random.randint(0,(h-th)) if h != th else 0
        j = random.randint(0,(w-tw)) if w != tw else 0
    
    return frames[:, i:i+th, j:j+tw, :] 


def center_crop(frames,out_size=(224,224)):
    t,h,w,c = frames.shape
    th,tw = out_size
    i = int(np.round((h - th) / 2.))
    j = int(np.round((w - tw) / 2.))
    return frames[:, i:i+th, j:j+tw, :]


def load_rgb_frames_from_video(video_path, start, num, resize=(256, 256)):
    vidcap = cv2.VideoCapture(video_path)

    frames = list()

    total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

    vidcap.set(cv2.CAP_PROP_POS_FRAMES, start)
    for offset in range(min(num, int(total_frames - start))):
      success, img = vidcap.read()
      if success:
        w, h, c = img.shape
        if w < 226 or h < 226:
            d = 226. - min(w, h)
            sc = 1 + d / min(w, h)
            img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)

        if w > 256 or h > 256:
            img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))

        img = (img / 255.) * 2 - 1
        # img = (img / 255.)
        frames.append(img)

    return np.asarray(frames, dtype=np.float32)


def make_dataset(split_file, video_path, mode,vid):
  '''
  split file : path of nslt_100.json
  mode : flow or rgb
  num_classes : should be for now 100
  '''
  
  with open(split_file, 'r') as f:
      data = json.load(f)

  count_skipping = 0
  

  src = 0

  if not os.path.exists(video_path):
      return ()

  num_frames = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT))

  if mode == 'flow':
      num_frames = num_frames // 2

  if num_frames - 0 < 9:
      print("Skip video ", vid)
      count_skipping += 1
      return ()

  # label = np.zeros((num_classes, num_frames), np.float32)

  # for l in range(num_frames):
  #     c_ = data[vid]['action'][0]
  #     label[c_][l] = 1
  if count_skipping :
      print("skipped")

  if len(vid) == 5:
      return (vid, src, 0, data[vid]['action'][2] - data[vid]['action'][1])
  elif len(vid) == 6:  ## sign kws instances
      return (vid, src, data[vid]['action'][1], data[vid]['action'][2] - data[vid]['action'][1])


def padding(imgs,total_frames):
    if imgs.shape[0] < total_frames:
            num_padding = total_frames - imgs.shape[0]

            if num_padding:
                prob = np.random.random_sample()
                if prob > 0.5:
                    pad_img = imgs[0]
                    pad = np.tile(np.expand_dims(pad_img, axis=0), (num_padding, 1, 1, 1))
                    padded_imgs = np.concatenate([imgs, pad], axis=0)
                else:
                    pad_img = imgs[-1]
                    pad = np.tile(np.expand_dims(pad_img, axis=0), (num_padding, 1, 1, 1))
                    padded_imgs = np.concatenate([imgs, pad], axis=0)
    else:
            padded_imgs = imgs

    return padded_imgs




def get_classes(num_classes,path):
    file = open(path,'r')
    all_lines = file.readlines()
    lines = all_lines[:num_classes]
    classes=list()
    for line in lines:
        label=line.split()[0]
        classes.append(label)
    return classes





def test_video(video_path):
  total_frames = 64
  cap = cv2.VideoCapture(video_path)
  nf = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  print(nf)
  start_frame = 0
  try:
    start_f = random.randint(0, nf - total_frames - 1) + start_frame
  except ValueError:
    start_f = start_frame
  imgs = load_rgb_frames_from_video(video_path, start_f, total_frames)
  imgs = padding(imgs , total_frames)
  frames = center_crop(imgs)
  return frames


Check The Number Of Samples in Each Split

In [ ]:
p = 'data100/'
folders = os.listdir("data100/")
for folder in folders:
  print(folder)
  count = 0
  classes = os.listdir(os.path.join(p,folder))
  for classs in classes:
    videos = len(os.listdir(os.path.join(p,folder,classs)))
    count += videos

  print(count) 

train
1264
val
298
test
235


In [ ]:
def generator_train():
      root_dir= 'data100/train/' #directory of the train folder
      classes= os.listdir(root_dir)
      classes = [int(x) for x in classes]
      classes.sort()
      classes = [str(x) for x in classes] 
      no_of_samples_in_each_class = np.array([len(os.listdir(os.path.join(root_dir,classs))) for classs in classes ])
      pointer_of_each_class = np.array([0 for _ in classes])
      samples_in_each_class= np.array([os.listdir(os.path.join(root_dir,classs)) for classs in classes])
      split_file = "code/I3D/preprocess/nslt_100.json"
      mode = "rgb"
      while True:
        ptr_sum=np.sum(pointer_of_each_class)
        samples_sum= np.sum(no_of_samples_in_each_class)
        if  ptr_sum == samples_sum: #terminating condition
            break

        # get random class
        while True: # loop until you get a class that has samples left
            current_class_indx= np.random.randint(0,len(classes)) # class index

            if pointer_of_each_class[current_class_indx] <= no_of_samples_in_each_class[current_class_indx]-1:
                break

        i = pointer_of_each_class[current_class_indx] # sample index
        pointer_of_each_class[current_class_indx]+=1 #increment the pointer

        c_name = classes[current_class_indx] # class name
        v_name= samples_in_each_class[current_class_indx][i] # video name in class i
        video_path = os.path.join(root_dir,c_name,v_name)

        # read frames from video ( adjust that to your fast code )
        vid = v_name[:-4]
        info  = make_dataset(split_file , video_path, mode , vid)
        if info:
          vid, src, start_frame , nf = info            
          total_frames = 64
          try:
            start_f = random.randint(0, nf - total_frames - 1) + start_frame
          except ValueError:
            start_f = start_frame
          imgs = load_rgb_frames_from_video(video_path, start_f, total_frames)
          imgs = padding(imgs , total_frames)
          frames = random_crop(imgs)
          flip_flag=0
          new_frames=[]
          if random.random() > 0.5:
            flip_flag=1
          if flip_flag:
            new_frames = np.flip(frames, axis=2).copy()
          else:
            new_frames = frames
        
          # yeild the video
          yield np.array(new_frames,dtype=np.float32) ,np.array([int(c_name)],dtype=np.int32)

def generator_val():
    root_dir= 'data100/val/' #directory of the val folder
    classes= os.listdir(root_dir)
    classes = [int(x) for x in classes]
    classes.sort()
    classes = [str(x) for x in classes] 
    no_of_samples_in_each_class = np.array([len(os.listdir(os.path.join(root_dir,classs))) for classs in classes ])
    pointer_of_each_class = np.array([0 for _ in classes])
    samples_in_each_class= np.array([os.listdir(os.path.join(root_dir,classs)) for classs in classes])
    split_file = "code/I3D/preprocess/nslt_100.json"
    mode = "rgb"
    while True:
        ptr_sum=np.sum(pointer_of_each_class)
        samples_sum= np.sum(no_of_samples_in_each_class)
        if  ptr_sum == samples_sum: #terminating condition
            break

        # get random class
        while True: # loop until you get a class that has samples left
            current_class_indx= np.random.randint(0,len(classes)) # class index

            if pointer_of_each_class[current_class_indx] <= no_of_samples_in_each_class[current_class_indx]-1:
                break

        i = pointer_of_each_class[current_class_indx] # sample index
        pointer_of_each_class[current_class_indx]+=1 #increment the pointer

        c_name = classes[current_class_indx] # class name
        v_name= samples_in_each_class[current_class_indx][i] # video name in class i
        video_path = os.path.join(root_dir,c_name,v_name)

        # read frames from video ( adjust that to your fast code )
        vid = v_name[:-4]
        info  = make_dataset(split_file , video_path, mode , vid)
        if info:
          vid, src, start_frame , nf = info            
          total_frames = 64
          try:
            start_f = random.randint(0, nf - total_frames - 1) + start_frame
          except ValueError:
            start_f = start_frame
          imgs = load_rgb_frames_from_video(video_path, start_f, total_frames)
          imgs = padding(imgs , total_frames)
          frames = center_crop(imgs)
          # yeild the video
          yield np.array(frames,dtype=np.float32) ,np.array([int(c_name)],dtype=np.int32)

In [ ]:

train_gen = tf.data.Dataset.from_generator(generator_train,
                                           output_signature=
                                           (tf.TensorSpec(shape=(64,224,224,3), dtype=tf.float32),tf.TensorSpec(shape=(1,),dtype=tf.int32)))

val_gen = tf.data.Dataset.from_generator(generator_val,
                                         output_signature=
                                         (tf.TensorSpec(shape=(64,224,224,3), dtype=tf.float32),tf.TensorSpec(shape=(1,),dtype=tf.int32)))


train_gen = train_gen.batch(6)
val_gen = val_gen.batch(6)
EPOCHS = 200

In [ ]:
if os.path.exists("/content/drive/MyDrive/WLASL/model/WLASL_drop_05_batch32_freezemodel.hdf5"):
  m_rgb = tf.keras.models.load_model("/content/drive/MyDrive/WLASL/model/WLASL_drop_05_batch32_freezemodel.hdf5")
else :
  m_rgb = PreTrainedInception3d(include_top=False, pretrained_weights="rgb_imagenet_and_kinetics", dropout_prob=0.5,
                        input_shape=(64, 224, 224, 3), classes=400)

  # m_rgb = layers_freeze(m_rgb)
  # print("Freezing layers done")
  m_rgb = add_top_layer(m_rgb, classes=100, dropout_prob=0.5)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-7)
m_rgb.compile(optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False) , metrics=['accuracy'])
  # model_ckpts = model_checkpoints()


In [ ]:
# m_rgb.trainable = True

In [ ]:
m_rgb.summary()

In [ ]:
model_ckpts = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/WLASL/model/WLASL_drop_05_batch32_freezemodel.hdf5",monitor='loss',mode="min",verbose=2,save_best_only=True)
m_rgb.fit( train_gen  , validation_data=val_gen , batch_size=32 ,verbose=1 ,epochs=EPOCHS ,callbacks=model_ckpts)

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


     40/Unknown - 517s 13s/step - loss: 4.7791 - accuracy: 0.0071

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


40/40 [==============================] - 643s 16s/step - loss: 4.7791 - accuracy: 0.0071 - val_loss: 4.6010 - val_accuracy: 0.0168

Epoch 00001: loss improved from inf to 4.77911, saving model to /content/drive/MyDrive/WLASL/model/WLASL_drop_05_batch32_freezemodel.hdf5
Epoch 2/200
40/40 [==============================] - 613s 15s/step - loss: 4.5538 - accuracy: 0.0285 - val_loss: 4.5280 - val_accuracy: 0.0336

Epoch 00002: loss improved from 4.77911 to 4.55385, saving model to /content/drive/MyDrive/WLASL/model/WLASL_drop_05_batch32_freezemodel.hdf5
Epoch 3/200
40/40 [==============================] - 612s 15s/step - loss: 4.4231 - accuracy: 0.0451 - val_loss: 4.4785 - val_accuracy: 0.0503

Epoch 00003: loss improved from 4.55385 to 4.42308, saving model to /content/drive/MyDrive/WLASL/model/WLASL_drop_05_batch32_freezemodel.hdf5
Epoch 4/200
26/40 [==================>...........] - ETA: 2:53 - loss: 4.2508 - accuracy: 0.0817

##**Loading Pretrained Weights Instead of Training**

In [ ]:
def get_classes(num_classes,path):
    file = open(path,'r')
    all_lines = file.readlines()
    lines = all_lines[:num_classes]
    classes=list()
    for line in lines:
        label=line.split()[1]
        classes.append(label)
    return classes

classes = get_classes(100,path="/content/drive/MyDrive/WLASL/code/I3D/preprocess/wlasl_class_list.txt")

Install onnx_tf

In [ ]:
git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e . 

To change the path to ../WLASL/code/I3D/
Run the following cell

In [ ]:
%cd ../WLASL/code/I3D

Important Imports

In [ ]:
import math
import os
import argparse
import torch
import sys
sys.path
sys.path.append('/content/drive/MyDrive/WLASL/onnx-tensorflow/')
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from torchvision import transforms
import videotransforms

import numpy as np

import torch.nn.functional as F
from pytorch_i3d import InceptionI3d

# from nslt_dataset_all import NSLT as Dataset
from datasets.nslt_dataset_all import NSLT as Dataset
import cv2

put the code of this cell in file code get_model.py and put this file in ../WLASL/code/I3D/


Then run 

!python get_model.py

In [ ]:
import math
import os
import argparse
import torch
import sys
sys.path
sys.path.append('/content/drive/MyDrive/WLASL/onnx-tensorflow/')
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from torchvision import transforms
import videotransforms

import numpy as np

import torch.nn.functional as F
from pytorch_i3d import InceptionI3d

# from nslt_dataset_all import NSLT as Dataset
from datasets.nslt_dataset_all import NSLT as Dataset
import cv2


weights = "/content/drive/MyDrive/WLASL/code/I3D/archived/asl100/FINAL_nslt_100_iters=896_top1=65.89_top5=84.11_top10=89.92.pt"

i3d = InceptionI3d(400, in_channels=3)
i3d.load_state_dict(torch.load('weights/rgb_imagenet.pt'))
i3d.replace_logits(100)
i3d.load_state_dict(torch.load(weights))


dummy_input = Variable(torch.randn(1,3,64,224,224))
torch.onnx.export(i3d, dummy_input, "i3d.onnx")

In [ ]:
 model = onnx.load('i3d.onnx')
 tf_rep = prepare(model)

Loading video sample to test the model

In [ ]:
video = test_video("/content/drive/MyDrive/test6.mp4")
video = np.moveaxis(video,-1,0)
video.shape
print(video.shape)

101
(64, 224, 224, 3)


In [ ]:
output = tf_rep.run(np.expand_dims(video,0))
result=np.argmax(np.max(np.array(output),axis=-1))
classes[result]

To Test The Whole Dataset :

In [ ]:
def generator_test():
    root_dir= '/content/drive/MyDrive/WLASL/data100/test' #directory of the test folder
    classes= os.listdir(root_dir)
    classes = [int(x) for x in classes]
    classes.sort()
    classes = [str(x) for x in classes] 
    no_of_samples_in_each_class = np.array([len(os.listdir(os.path.join(root_dir,classs))) for classs in classes ])
    pointer_of_each_class = np.array([0 for _ in classes])
    samples_in_each_class= np.array([os.listdir(os.path.join(root_dir,classs)) for classs in classes])
    split_file = "/content/drive/MyDrive/WLASL/code/I3D/preprocess/nslt_100.json"
    mode = "rgb"
    while True:
        ptr_sum=np.sum(pointer_of_each_class)
        samples_sum= np.sum(no_of_samples_in_each_class)
        if  ptr_sum == samples_sum: #terminating condition
            break

        # get random class
        while True: # loop until you get a class that has samples left
            current_class_indx= np.random.randint(0,len(classes)) # class index

            if pointer_of_each_class[current_class_indx] <= no_of_samples_in_each_class[current_class_indx]-1:
                break

        i = pointer_of_each_class[current_class_indx] # sample index
        pointer_of_each_class[current_class_indx]+=1 #increment the pointer

        c_name = classes[current_class_indx] # class name
        v_name= samples_in_each_class[current_class_indx][i] # video name in class i
        video_path = os.path.join(root_dir,c_name,v_name)

        # read frames from video ( adjust that to your fast code )
        vid = v_name[:-4]
        info  = make_dataset(split_file , video_path, mode , vid)
        if info:
          vid, src, start_frame , nf = info            
          total_frames = 64
          try:
            start_f = random.randint(0, nf - total_frames - 1) + start_frame
          except ValueError:
            start_f = start_frame
          imgs = load_rgb_frames_from_video(video_path, start_f, total_frames)
          imgs = padding(imgs , total_frames)
          frames = center_crop(imgs)
          # yeild the video
          yield np.array(frames,dtype=np.float32) ,np.array([int(c_name)],dtype=np.int32)

In [ ]:
counter = 0
i = 0
gen = generator_test()
for frames ,label in gen:
  frames = np.moveaxis(frames,-1,0)
  output = tf_rep.run(np.expand_dims(frames,0))
  result = np.argmax(np.max(np.array(output),axis=-1))
  if result == label:
    counter += 1
  i+=1
print(counter)
print(i)
print(counter / i)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


990
1264
0.7832278481012658
